# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [140]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')

### Question 0 (Example)

What is the first country in df? *This function should return a Series.*



In [1]:
def answer_zero():
    return df.iloc[0]

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [30]:
def answer_one():
    return df.loc[df['Gold'] == df['Gold'].max()].iloc[0].name

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [58]:
def answer_two():
    df['winter_summer_gold_diff'] = (df['Gold'].fillna(0) - df['Gold.1'].fillna(0))
    df['winter_summer_gold_diff'] = df['winter_summer_gold_diff'].abs()
    return df.loc[df['winter_summer_gold_diff'] == df['winter_summer_gold_diff'].max()].iloc[0].name

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [146]:
def answer_three():
    new_df = df.loc[df['Gold'] != 0]
    new_df = new_df.loc[df['Gold.1'] != 0]
    new_df['winter_summer_gold_diff'] = (new_df['Gold'].fillna(0) - new_df['Gold.1'].fillna(0))
    new_df['winter_summer_gold_diff'] = new_df['winter_summer_gold_diff'].abs()
    new_df['winter_summer_gold_diff_ratio'] = new_df['winter_summer_gold_diff'].divide(new_df[['Gold', 'Gold.1', 'Gold.2']].fillna(0).sum(axis=1))
    return new_df.loc[new_df['winter_summer_gold_diff_ratio'] == new_df['winter_summer_gold_diff_ratio'].max()].iloc[0].name

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [66]:
def answer_four():
    df['question_4'] = df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']
    return df['question_4']

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [182]:
census_df = pd.read_csv('census.csv')
new_df = census_df.loc[census_df['SUMLEV'] == 50]
new_df = new_df.set_index('COUNTY').groupby('STNAME')['CENSUS2010POP'].nlargest(3).reset_index()
new_df.sort_values('STNAME').head()

,STNAME,COUNTY,CENSUS2010POP
0,Alabama,73,658466
1,Alabama,97,412992
2,Alabama,89,334811
3,Alaska,20,291826
4,Alaska,90,97581
5,Alaska,170,88995
6,Arizona,13,3817117
7,Arizona,19,980263
8,Arizona,21,375770
9,Arkansas,119,382748


In [120]:
def answer_five():
    new_df = census_df.loc[census_df['SUMLEV'] == 50, ['SUMLEV', 'STNAME']].groupby('STNAME').count()
    return new_df[new_df['SUMLEV'] == new_df['SUMLEV'].max()].iloc[0].name

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [184]:
def answer_six():
    new_df = census_df.loc[census_df['SUMLEV'] == 50]
    new_df = new_df.set_index('COUNTY').groupby('STNAME')['CENSUS2010POP'].nlargest(3).reset_index()
    new_df = new_df.groupby('STNAME')[['CENSUS2010POP']].sum().reset_index()
    
    return new_df.sort_values('CENSUS2010POP', ascending=False).head(3)['STNAME'].tolist()
answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [175]:
def answer_seven():
    new_df = census_df.loc[census_df['SUMLEV'] == 50]
    new_df['POP_MAX'] = new_df[['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']].max(axis=1)
    new_df['POP_MIN'] = new_df[['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']].min(axis=1)
    new_df['POP_CHANGE'] = (new_df['POP_MAX']-new_df['POP_MIN']).abs()
    return new_df.loc[new_df['POP_CHANGE'] == new_df['POP_CHANGE'].max(), 'CTYNAME'].iloc[0]

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [137]:
def answer_eight():
    region_filter = census_df.loc[census_df['REGION'].isin([1,2])]
    washington_filter = region_filter.loc[region_filter['CTYNAME'].str.startswith('Washington')]
    pop_filter = washington_filter.loc[washington_filter['POPESTIMATE2015'] > washington_filter['POPESTIMATE2014']]
    return pop_filter[['STNAME', 'CTYNAME']].sort_index()